<a href="https://colab.research.google.com/github/Omar-GIT-Portfollio/Python-Projects/blob/main/calculate_the_area_in_square_miles_of_four_selected_U_S_counties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Omar Ali   
##11/10/25


#Purpose:
To calculate the area in square miles of four selected U.S. counties using the same sinusoidal projection and area computation methods we learned in lecture and the shapefile tutorial.

#Counties:  
Franklin County, OH · Augusta County, VA · Aleutians West Census Area, AK · Los Angeles County, CA  

#Method:
1. Read the shapefile cb_2020_us_county_5m.shp from the Census Cartographic Boundary Files.  
2. Project coordinates to sinusoidal (unit sphere) using transform_sinusoidal.  
3. Convert degrees to miles using the constant \(3959 \times \pi / 180\).  
4. Apply the shoelace formula on projected polygons to compute the area.


 First im going to set up my notebook to make sure everything runs correctly

In [ ]:
!git clone https://github.com/gisalgs/geom.git

Cloning into 'geom'...
remote: Enumerating objects: 378, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 378 (delta 29), reused 6 (delta 2), pack-reused 314 (from 1)
Receiving objects: 100% (378/378), 100.03 KiB | 2.04 MiB/s, done.
Resolving deltas: 100% (206/206), done.


In [ ]:
from geom.shapex import*

In [ ]:
shp =shapex('Data/uscnty48area.shp')

In [ ]:
shp

In [ ]:
shpj = shp2geojson(shp)

In [ ]:
shpj['type']

'FeatureCollection'

In [ ]:
### I used the same libraries we used in class
### - geopandas for reading shapefiles
### - shapely for polygon geometry (Polygon, MultiPolygon)
### - math for constants like pi
### - transform_sinusoidal() from transforms.py
import math
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from transforms import transform_sinusoidal

In [ ]:
### Step 1: Read Census counties (cb_2020_us_county_5m.shp)
### - Used a similar command like we did in class to read GeoJSONs and shapefiles.
### - Important fields: NAMELSAD (county name) and STATE_NAME (state).
import os, geopandas as gpd

DATA_DIR = "/content/Data"
base = "cb_2020_us_county_5m"

SHP = os.path.join(DATA_DIR, base + ".shp")
counties = gpd.read_file(SHP)



In [ ]:
### Step 2: Project coordinates & compute area
### What we practiced in lecture and tut:
###  - transform_sinusoidal(lon, lat, lon0) to projects to unit sphere
###  - multiply by (3959 * pi / 180) to converts degrees to miles
###  - apply shoelace formula to compute polygon area
###  - subtract holes and handle multipolygons (like Aleutians West)

DEG_TO_MILES = 3959.0 * math.pi / 180.0  # miles per degree on great circle

def project_to_miles(lon, lat, lon0=0.0): # Project lon/lat to sinusoidal (unit sphere) then scale to miles.
    x_deg, y_deg = transform_sinusoidal(lon, lat, lon0)
    return x_deg * DEG_TO_MILES, y_deg * DEG_TO_MILES

def ring_area_mi2(coords):   # Shoelace on a single ring after projecting each vertex to miles. #  close ring if needed
    xs, ys = [], []
    for lon, lat in coords:
        x, y = project_to_miles(lon, lat, 0.0)
        xs.append(x); ys.append(y)
    if (xs[0], ys[0]) != (xs[-1], ys[-1]):
        xs.append(xs[0]); ys.append(ys[0])
    s = 0.0
    for i in range(len(xs)-1):
        s += xs[i]*ys[i+1] - xs[i+1]*ys[i]
    return 0.5*s

def polygon_area_mi2(poly: Polygon) -> float:      # Exterior minus holes all in square miles.
    a = abs(ring_area_mi2(poly.exterior.coords))
    for hole in poly.interiors:
        a -= abs(ring_area_mi2(hole.coords))
    return a

def area_mi2(geom) -> float:              # Polygon or MultiPolygon to area in square miles.
    if isinstance(geom, Polygon):
        return polygon_area_mi2(geom)
    if isinstance(geom, MultiPolygon):
        return sum(polygon_area_mi2(p) for p in geom.geoms)
    raise TypeError("Expected Polygon or MultiPolygon.")

In [ ]:
### Step 3: Select counties by name
### - Match on both NAMELSAD and STATE_NAME fields.
### - This ensures we select the correct feature for each county.


def get_geom(namelsad, state_name):
    sel = counties[
        (counties["NAMELSAD"].str.upper() == namelsad.upper()) &
        (counties["STATE_NAME"].str.upper() == state_name.upper())
    ]
    assert not sel.empty, f"Not found: {namelsad}, {state_name}"
    return sel.iloc[0].geometry

targets = [
    ("Franklin County", "Ohio"),
    ("Augusta County", "Virginia"),
    ("Aleutians West Census Area", "Alaska"),
    ("Los Angeles County", "California"),  # my choice
]

In [ ]:
### Step 4: Compute and print the results
### - Each line reports area in square miles rounded to two decimals.
for cname, sname in targets:
    geom = get_geom(cname, sname)
    a = area_mi2(geom)
    print(f"The area of {cname}, {sname} is {a:,.2f} square miles.")


The area of Franklin County, Ohio is 543.70 square miles.
The area of Augusta County, Virginia is 970.22 square miles.
The area of Aleutians West Census Area, Alaska is 4,783.40 square miles.
The area of Los Angeles County, California is 4,099.57 square miles.
